In [ ]:
import logging
import math
import os
import sys
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Callable, Optional, List
import json
import jsonlines
import shutil
import numpy as np
import torch
import clip
import matplotlib.pyplot as plt 
from tqdm.auto import tqdm
from PIL import Image

#import transformers
'''
from transformers import (
    CONFIG_MAPPING,
    AutoConfig,
    FlaxCLIPModel,
    CLIPModel,
    CLIPProcessor,
    CLIPTokenizer,
    TrainingArguments,
    is_tensorboard_available,
    IntervalStrategy
    
)
'''
from torchvision.datasets import VisionDataset
#from transformers.testing_utils import CaptureLogger
#from torchvision.io import ImageReadMode, read_image
'''
from torchvision.transforms import (
    # added for image augmentation
    ToPILImage,
    RandomCrop,
    ColorJitter,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomResizedCrop,
    ToTensor,
    # /added for image augmentation
    CenterCrop, 
    ConvertImageDtype, 
    Normalize, 
    Resize
)
'''
#from torchvision.transforms.functional import InterpolationMode

import jsonlines
from pathlib import Path
from typing import Optional, Callable
from importlib.util import find_spec
import torch.nn as nn
logger = logging.getLogger(__name__)

In [ ]:
sys.path.append(r"./data/image_dataset/train_images")

In [ ]:
from fastai.vision.all import *
from fastai.distributed import *

In [ ]:
from self_supervised.multimodal.clip import *
import clip

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    dtype: Optional[str] = field(
        default="float32",
        metadata={
            "help": "Floating-point format in which the model weights should be initialized and trained. Choose one of `[float32, float16, bfloat16]`."
        },
    )
    save_optimizer: Optional[bool] = field(
        default=True,
        metadata={"help": "Whether to store full train state including optimizer."},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    data_dir: Optional[str] = field(
        default=None, metadata={"help": "Path to local folder containing data files."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file (a jsonlines file)."},
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Optional input sequence length after tokenization. "
            "The training dataset will be truncated in block of this size for training. "
            "Default to the model max input length for single sentence inputs (take into account special tokens)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    text_column_name: Optional[str] = field(
            default='text',
            metadata={"help": "Column containing main text data."},
    )
    augment_images: Optional[bool] = field(
        default=True,
        metadata={ "help": "Augment input training images" }
    )
    augment_captions: Optional[bool] = field(
        default=True,
        metadata={ "help": "Augment input training images" }
    )
    captions_per_image: Optional[int] = field(
        default=5,
        metadata={"help": "Number of captions per image to use when creating train dataset."},
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt", "jsonl"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt", "jsonl"], "`validation_file` should be a csv, a json or a txt file."


@dataclass
class ImageAugmentationArguments:
    """
    Arguments for image augmentations configuration
    """
    random_horizontal_flip: Optional[float] = field(
        default=0.5,
        metadata={ "help": "Probability of applying random horizontal flip" }
    )
    random_vertical_flip: Optional[float] = field(
        default=0.5,
        metadata={ "help": "Probability of applying random vartical flip" }
    )


In [ ]:
data_args = DataTrainingArguments(
    dataset_name = None,
    data_dir = "./data",
    train_file = "data/train_lines.jsonl",
    validation_file = "data/train_lines.jsonl",
    max_train_samples = None,
    max_eval_samples = None,
    overwrite_cache = False,
    validation_split_percentage = 5,
    preprocessing_num_workers = 0,
    augment_images = False, ##Can be changed to augment captions and images (NEEDS FUTURE WORK)
    augment_captions = False,
    captions_per_image = 5,
)

augmentation_args = ImageAugmentationArguments()

In [ ]:
class ImageTextDataset(VisionDataset):
    """
    Dtaset for loading image-text data for tasks like CLIP training, Image Captioning.

    Args:
        root: (string): The root path where the dataset is stored.
            The expected format is jsonlines where each line is a json object containing to keys.
            `filename`: The path to the image.
            `captions`: An `array` of captions.
        split: (string): Dataset split name. Is used for parsing jsonl files from `root` folder.
        captions_per_image: (int): number of captions per image to use. Defaults to 5.
        augment_captions: (bool): If true the jsonl files with `textaug_` prefix are selected from root
            folder. 
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.ToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(
        self,
        root: str,
        split: str, 
        captions_per_image:int = 5,
    ):
        super().__init__(root, None, None, None)
        self.root = root
        prefix = ""
        filepaths = list(Path(root).glob(f"{prefix}train*.jsonl"))
        fps_empty_msg = f"""\
        The `filepaths` is empty. Please make sure that `root` folder contains jsonl files
        named properly: [textaug_]{split}*.jsonl.
        `textaug_` prefix is expected if `augment_captions` is `True`.
        """
        assert len(filepaths) > 0, fps_empty_msg
        self.captions_dict = {}
        self.captions = []
        self.image_paths = []
        for count, filepath in enumerate(filepaths):
            with jsonlines.open(filepath, "r") as reader:
                length = len(list(jsonlines.open(filepath, "r")))
                step =0 
                for example in reader:
                    if captions_per_image ==1:
                        captions_per_image_cur = 1
                    else:
                        captions_per_image_cur = len(example["Captions"])
                    
                    if split=='train' and step < 0.01*length:
                        self.captions.extend(example["Captions"][:captions_per_image_cur])
                        tmp_dict = {example["URL"]:example["Captions"][:captions_per_image_cur] }
                        self.image_paths.extend([example["URL"]] * captions_per_image_cur)
                        self.captions_dict.update(tmp_dict)
                    elif split=='valid' and step > 0.99*length:
                        self.captions.extend(example["Captions"][:captions_per_image_cur])
                        self.image_paths.extend([example["URL"]] * captions_per_image_cur)
                    step+=1
        print(f"{count+1} input files for {split} split found")
    
    def _load_image(self, idx: int):
        path = f"{self.root}/image_dataset/train_images/{self.image_paths[idx]}"
        return preprocess(PILImage.create(path))
        #return preprocess(Image.open(path))
    
    def _load_target(self, idx):
        return self.captions[idx]

    def __getitem__(self, index: int):
        path = f"{self.root}/image_dataset/train_images/{self.image_paths[index]}"
        image = preprocess(Image.open(path))
        target = self.captions[index]

        if self.transforms is not None:
            image, target = self.transforms(image, target)
        return image, target

    def __len__(self) -> int:
        return len(self.captions)

In [ ]:
torch.cuda.is_available()
device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.

In [ ]:
device

In [ ]:
train_dataset = ImageTextDataset(
    data_args.data_dir,
    'train',
    captions_per_image=data_args.captions_per_image,
)


eval_dataset = ImageTextDataset(
    data_args.data_dir,
    'valid',
    captions_per_image=1,
)


In [ ]:
train_paths = train_dataset.image_paths
eval_paths = eval_dataset.image_paths

In [ ]:
train_captions = train_dataset.captions
eval_captions = eval_dataset.captions

In [ ]:
def read_image(fn): return PILImage.create(fn)
def read_text(fn): return full_captions[fn.name][np.random.randint(len(full_captions[fn.name]))]
def dummy_targ(o): return 0 

In [ ]:
def read_image_train(idx): return PILImage.create(f"./data/image_dataset/train_images/{train_paths[idx]}")
def read_text_train(idx): return train_captions[idx]
def dummy_targ(o): return 0 

In [ ]:
def read_image_valid(idx): return PILImage.create(f"./data/image_dataset/train_images/{eval_paths[idx]}")
def read_text_valid(idx): return eval_captions[idx]

In [ ]:
train_images = get_image_files(f"./data/image_dataset/train_images/")
#valid_images = get_image_files(validpath)

In [ ]:
clip_stats = ([0.48145466, 0.4578275, 0.40821073], [0.26862954, 0.26130258, 0.27577711])
clip_tokenizer = ClipTokenizer()
size,bs = 224, 16


item_tfms = [RandomResizedCrop(size, min_scale=0.9), clip_tokenizer, ToTensor()]

dsets_train = Datasets(np.arange(len(train_paths)), tfms=[read_image_train, read_text_train, dummy_targ],n_inp=2)
dsets_valid = Datasets(np.arange(len(eval_paths)), tfms=[read_image_valid, read_text_valid, dummy_targ],n_inp=2)

'''
n=int(0.85*len(train_images))
dsets_train = Datasets(train_images[:n], tfms=[read_image, read_text, dummy_targ],n_inp=2, get_idxs=None)
dsets_valid = Datasets(train_images[n:], tfms=[read_image, read_text, dummy_targ],n_inp=2, get_idxs=None)
'''

batch_tfms = [IntToFloatTensor, Normalize.from_stats(*clip_stats)]
train_dl = TfmdDL(dsets_train, shuffle=True, bs=bs, after_item=item_tfms, after_batch=batch_tfms, drop_last=True)

valid_dl = TfmdDL(dsets_valid, shuffle=False, bs=bs*2, after_item=item_tfms, after_batch=batch_tfms)

dls = DataLoaders(train_dl,valid_dl, device=default_device())

In [ ]:
for i in range(5): dls.train_ds[i][0].show(title=dls.train_ds[i][1])

In [ ]:
clip_trainer_cb = CLIPTrainer()
cbs = [clip_trainer_cb]
opt_func = ranger

arch = 'vitl14'
do_finetune = True
use_grad_check = True
grad_check_nchunks = 2
finetune_modelname = 'ViT-L/14'
#finetune_modelname = 'ViT-B/32'

#Custom CONFIG  for ViT-B/16
def vitb16_config(input_res, context_length, vocab_size):
    "ViT-B/16 configuration, uses 16x16 patches"
    return dict(embed_dim=512,
                image_resolution=input_res,
                vision_layers=12,
                vision_width=768,
                vision_patch_size=16,
                context_length=context_length,
                vocab_size=vocab_size,
                transformer_width=512,
                transformer_heads=8,
                transformer_layers=12)

#This portion was wrong in Turgutlu's implementation
def vitl14_config(input_res, context_length, vocab_size):
    "ViT-L/14 configuration, uses 14x14 patches"
    return dict(embed_dim=768,
                image_resolution=input_res,
                vision_layers=24,
                vision_width=1024,
                vision_patch_size=14,
                context_length=context_length,
                vocab_size=vocab_size,
                transformer_width=768,
                transformer_heads=8,
                transformer_layers=12)
vitb32_config_dict = vitb32_config(size, clip_tokenizer.context_length, clip_tokenizer.vocab_size)
#vitb32_config_dict = vitb32_config(size, clip_tokenizer.context_length, clip_tokenizer.vocab_size)

clip_model = CLIP(**vitb32_config_dict, checkpoint=use_grad_check, checkpoint_nchunks=grad_check_nchunks)

In [ ]:
if do_finetune:
    print("Loading pretrained model..")
    clip_pretrained_model, _ = clip.load(finetune_modelname, jit=False)
    clip_model.load_state_dict(clip_pretrained_model.state_dict())

In [ ]:
learner = Learner(dls, clip_model, loss_func=noop, cbs=cbs, opt_func=opt_func)
learner.to_fp16();

In [ ]:
learner.fit_flat_cos(1, 1e-5, pct_start=0.25)

In [ ]:
torch.save({
        'model_state_dict': clip_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_checkpoint/model_1_epoch{epoch}.pt") #just change to your preferred folder/filename

In [ ]:
learner.save(file ="./model_fastai_v14", with_opt=True, pickle_protocol=2)

In [ ]:
checkpoint = torch.load('./models/model_fastai.pth')

In [ ]:
clip_pretrained_model.load_state_dict(checkpoint['model'])

In [ ]:
image = _(Image.open('data/image_dataset/test_images/3625376_1dfc183f7b_z.jpg')).unsqueeze(0).to(device)
text = clip.tokenize(["a black and white dog holding a ball near sea"]).to(device)

with torch.no_grad():
    image_features = clip_pretrained_model.encode_image(image)
    text_features = clip_pretrained_model.encode_text(text)
    
    logits_per_image, logits_per_text = clip_pretrained_model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()
for l, p in zip(["A black dog","a white and black dog ","a black cat","a black and white dog near sea","a black and white dog holding a ball near sea"], probs[0]):
    print(f"{l:<16} {p:.4f}")

In [ ]:
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features @ image_features.T